In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/esoh/"
oCV_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

from scipy.io import savemat


In [2]:
## Added from old vrsion 


In [3]:
parameter_values = get_parameter_values()

In [4]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
# spm.print_parameter_info()
param=spm.param

In [5]:
cell = 7

In [6]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [7]:
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

# drive_cycle = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_default_5Ah.csv", comment="#", header=None).to_numpy()
# drive_cycle2 = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_GM_5Ah.csv", comment="#", header=None).to_numpy()

# experiment = pybamm.Experiment( # defult D.C. DRC-DRC *1.25
#     [
#         ("Run US06 (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Discharge at C/4 for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/4 until 4.2V",
#          "Hold at 4.2V for .9 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes", 
#          # Home
#          "Run US06 (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Discharge at C/4 for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/8 until 4.2V",
#           "Hold at 4.2V for 1.9 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes",     
#         )

#     ]*540,
#     drive_cycles={"US06": drive_cycle},
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

experiment2 = pybamm.Experiment( #lab cycling
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
#     ] *dfe.N.iloc[-1],
    ] *200,
    
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

## GM_DCRDCR
# experiment3 = pybamm.Experiment( #GM_ DRC-DRC
#     [
#         ("Run US06_GM_5Ah (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Discharge at C/4 for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/4 until 4.2V",
#          "Hold at 4.2V for .6 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes", 
#          # Home
#          "Run US06_GM_5Ah (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Discharge at C/4 for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/8 until 4.2V",
#           "Hold at 4.2V for 1 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes",     
#         )

#     ]*540,
#     drive_cycles={"US06_GM_5Ah": drive_cycle2},
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

# experiment4 = pybamm.Experiment( #Gm-RCRC
#     [
#         ("Run US06_GM_5Ah (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Rest for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/4 until 4.2V",
#          "Hold at 4.2V for 1.6 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes", 
#          # Home
#          "Run US06_GM_5Ah (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Rest for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/8 until 4.2V",
#           "Hold at 4.2V for 3.1 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes",     
#         )

#     ]*540,
#     drive_cycles={"US06_GM_5Ah": drive_cycle2},
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

# experiment5 = pybamm.Experiment( # defult D.C. RC-RC
#     [
#         ("Run US06 (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Rest for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/4 until 4.2V",
#          "Hold at 4.2V for 1.8 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes", 
#          # Home
#          "Run US06 (A) for 1 hours",
#          "Rest for 5 minutes",
#          "Rest for 1 hours", # should be P/4 
#          "Rest for 5 minutes",
#          "Charge at C/8 until 4.2V",
#           "Hold at 4.2V for 3.7 hours",  # needs fixing : last step +this step =2.5 hrs
#          "Rest for 20 minutes",     
#         )

#     ]*540,
#     drive_cycles={"US06": drive_cycle},
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

In [8]:
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]": 2.5112e-07,
        "Negative electrode LAM constant proportional term [s-1]": 2.4741e-06,
        "Positive electrode LAM constant exponential term": 1.1942,
        "Negative electrode LAM constant exponential term": 1.1942,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
    },
    check_already_exists=False,
)


In [9]:
# all_sumvars_dict1 = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [15]:
parameter_values.search("exchange")

Exchange-current density for plating [A.m-2]	<function plating_exchange_current_density_Mohtat2020 at 0x00000246CE7BBA60>
Exchange-current density for stripping [A.m-2]	<function stripping_exchange_current_density_Mohtat2020 at 0x00000246CE7BB670>
Negative electrode exchange-current density [A.m-2]	<function graphite_electrolyte_exchange_current_density_PeymanMPM at 0x00000246CE7FB160>
Negative electrode reference exchange-current density [A.m-2(m3.mol)1.5]	4.244e-06
Positive electrode exchange-current density [A.m-2]	<function NMC_electrolyte_exchange_current_density_PeymanMPM at 0x00000246CE7FB5E0>
Positive electrode reference exchange-current density [A.m-2(m3.mol)1.5]	4.824e-06
SEI reaction exchange current density [A.m-2]	1.5e-07


In [10]:
sim_long = pybamm.Simulation(spm, experiment=experiment2, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe", dt_max=1))
sol_long = sim_long.solve(initial_soc=SOC_0, save_at_cycles=10  )

2023-01-17 10:41:40.762 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/200 (18.611 ms elapsed) --------------------
2023-01-17 10:41:40.762 - [NOTICE] callbacks.on_step_start(182): Cycle 1/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:40.936 - [NOTICE] callbacks.on_step_start(182): Cycle 1/200, step 2/4: Rest for 5 min
2023-01-17 10:41:41.022 - [NOTICE] callbacks.on_step_start(182): Cycle 1/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:41.168 - [NOTICE] callbacks.on_step_start(182): Cycle 1/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:41.626 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.848 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:41.626 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/200 (883.846 ms elapsed) --------------------
2023-01-17 10:41:41.626 - [NOTICE] callbacks.on_step_start(182): Cycle 2/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:41.710 - [NOTICE] callbacks.on_step_start(182): Cycle 2/2

2023-01-17 10:41:44.996 - [NOTICE] callbacks.on_cycle_start(174): Cycle 13/200 (4.247 s elapsed) --------------------
2023-01-17 10:41:44.996 - [NOTICE] callbacks.on_step_start(182): Cycle 13/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:45.077 - [NOTICE] callbacks.on_step_start(182): Cycle 13/200, step 2/4: Rest for 5 min
2023-01-17 10:41:45.092 - [NOTICE] callbacks.on_step_start(182): Cycle 13/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:45.163 - [NOTICE] callbacks.on_step_start(182): Cycle 13/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:45.301 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.775 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:45.301 - [NOTICE] callbacks.on_cycle_start(174): Cycle 14/200 (4.560 s elapsed) --------------------
2023-01-17 10:41:45.301 - [NOTICE] callbacks.on_step_start(182): Cycle 14/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:45.387 - [NOTICE] callbacks.on_step_start(182): Cycle 1

2023-01-17 10:41:48.477 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.703 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:48.477 - [NOTICE] callbacks.on_cycle_start(174): Cycle 25/200 (7.734 s elapsed) --------------------
2023-01-17 10:41:48.477 - [NOTICE] callbacks.on_step_start(182): Cycle 25/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:48.540 - [NOTICE] callbacks.on_step_start(182): Cycle 25/200, step 2/4: Rest for 5 min
2023-01-17 10:41:48.550 - [NOTICE] callbacks.on_step_start(182): Cycle 25/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:48.614 - [NOTICE] callbacks.on_step_start(182): Cycle 25/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:48.747 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.696 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:48.747 - [NOTICE] callbacks.on_cycle_start(174): Cycle 26/200 (8.006 s elapsed) --------------------
2023-01-17 10:41:48.747 - [NOTICE] callbacks.on_step

2023-01-17 10:41:51.743 - [NOTICE] callbacks.on_step_start(182): Cycle 36/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:51.894 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.621 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:51.894 - [NOTICE] callbacks.on_cycle_start(174): Cycle 37/200 (11.146 s elapsed) --------------------
2023-01-17 10:41:51.894 - [NOTICE] callbacks.on_step_start(182): Cycle 37/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:51.960 - [NOTICE] callbacks.on_step_start(182): Cycle 37/200, step 2/4: Rest for 5 min
2023-01-17 10:41:51.975 - [NOTICE] callbacks.on_step_start(182): Cycle 37/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:52.027 - [NOTICE] callbacks.on_step_start(182): Cycle 37/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:52.170 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.614 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:52.170 - [NOTICE] callbacks.on_cycle_st

2023-01-17 10:41:55.275 - [NOTICE] callbacks.on_step_start(182): Cycle 48/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:55.325 - [NOTICE] callbacks.on_step_start(182): Cycle 48/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:55.468 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.535 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:41:55.468 - [NOTICE] callbacks.on_cycle_start(174): Cycle 49/200 (14.727 s elapsed) --------------------
2023-01-17 10:41:55.468 - [NOTICE] callbacks.on_step_start(182): Cycle 49/200, step 1/4: Discharge at 2C until 3V
2023-01-17 10:41:55.535 - [NOTICE] callbacks.on_step_start(182): Cycle 49/200, step 2/4: Rest for 5 min
2023-01-17 10:41:55.556 - [NOTICE] callbacks.on_step_start(182): Cycle 49/200, step 3/4: Charge at 2C until 4.2V
2023-01-17 10:41:55.610 - [NOTICE] callbacks.on_step_start(182): Cycle 49/200, step 4/4: Hold at 4.2V until C/50
2023-01-17 10:41:55.752 - [NOTICE] callbacks.on_cycle_end(196): Capacity is n

...

psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../c

psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../c

psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../c

psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../casadi/interfaces/sundials/cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: .../c

2023-01-17 10:42:38.703 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.720 Ah (originally 4.848 Ah, will stop at 2.424 Ah)
2023-01-17 10:42:38.709 - [NOTICE] callbacks.on_experiment_end(222): Finish experiment simulation, took 57.955 s


In [11]:
qwrqwf

NameError: name 'qwrqwf' is not defined

In [ ]:
pybamm.plot_summary_variables(sol_long)

In [12]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries
Epsilon_n_s_cycled=sol_long["X-averaged negative electrode active material volume fraction"].entries
Epsilon_p_s_cycled=sol_long["X-averaged positive electrode active material volume fraction"].entries

# ECM_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]

# ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
# ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
# ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']

cycle_time=sol_long.summary_variables['Time [s]'],
Vt_t = sol_long["Terminal voltage [V]"].entries,

LLI_SEI_mol=sol_long["Loss of lithium to SEI [mol]"].entries,
LLI_plating_mol=sol_long["Loss of lithium to lithium plating [mol]"].entries,

In [13]:
ident=""
mdic = {
    "x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,    
#         "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
#         "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time 
        "Epsilon_n_s_cycled"+ident:Epsilon_n_s_cycled,
        "Epsilon_p_s_cycled"+ident:Epsilon_p_s_cycled,
        "cycle_time":cycle_time, "Vt_t":Vt_t,
        "LLI_SEI_mol":LLI_SEI_mol,
        "LLI_plating_mol":LLI_plating_mol,
    
       }
savemat("experiment2_jan17_cell07.mat",mdic)

In [ ]:
sim_long.plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#         "Electrolyte concentration [mol.m-3]",
#         "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
#         "Negative electrode potential [V]",
#         "Electrolyte potential [V]",
#         "Positive electrode potential [V]",
        "Terminal voltage [V]",
#         "X-averaged cell temperature",
        "Negative electrode SOC",
        "Positive electrode SOC",
#         "SEI thickness [m]",
        "Loss of capacity to SEI [A.h]",
        "Negative electrode capacity [A.h]",
        "Positive electrode capacity [A.h]",
        "Discharge capacity [A.h]"

        
    ]
)

In [ ]:
print(list(sol_long.summary_variables.keys()))

In [ ]:
type(sol_long.summary_variables[list(sol_long.summary_variables.keys())[1]])

In [ ]:
# mdic={}
# for i in range (1):
# # range(len(sol_long.summary_variables)):
# #     stringing=''.join([list(sol_long.summary_variables.keys())[i]])
#     mdic={f"{i}": sol_long.summary_variables[list(sol_long.summary_variables.keys())[i]]}
# savemat("summary",mdic)


In [ ]:
ident=""

In [ ]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries

ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']
cycle_time=sol_long.summary_variables['Time [s]']
# exp = 30e6*sol_0["Cell thickness change [m]"].entries
# exp_n = 30e6*sol_0["Negative Electrode thickness change [m]"].entries
# exp_p = 30e6*sol_0["Positive Electrode thickness change [m]"].entries
# ee_p = sol_0["X-averaged positive electrode porosity"].entries
# ee_n = sol_0["X-averaged negative electrode porosity"].entries
# ee_s = sol_0["X-averaged separator porosity"].entries
# j_n = sol_0["X-averaged negative electrode total interfacial current density [A.m-2]"].entries
# j_p = sol_0["X-averaged positive electrode total interfacial current density [A.m-2]"].entries

# x = sol_0["x [m]"].entries
# c_e_0 = sol_0["Electrolyte concentration [mol.m-3]"].entries

In [ ]:
mdic = {"x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,    
        "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
        "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time 
       }
savemat("Outputs_Oct23_GM_DRC_DRC",mdic)

In [ ]:
# savemat("x100_test", mdic)


In [ ]:
# print(x100)

In [ ]:
# import csv
# with open(r"F:\Git Hub rep\CRC\neew\PyBaMM\Untitled Folder\whatever.csv", 'w') as f:
#     # create the csv writer
#     writer = csv.writer(f)

# write a row to the csv file
# i=1
# for lines in sol_long.all_summary_variables:
# #     writer.writerow(words)
#     i=i+1
#     savemat(f'{i:02d}', lines)


In [ ]:
type(x100)

In [ ]:
sol_long.variables.search("charge")

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Current [A]"].entries[1:15133])

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Terminal voltage [V]"].entries[1:15133])

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long["Time [s]"].entries

In [ ]:
spm.variables.search("resistance")

In [ ]:
sim_long.plot(["Local ECM resistance",
"Local ECM resistance [Ohm]",
               "Resistance [Ohm]"])


In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long.summary_variables 